In [116]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import torch
import random

In [117]:
df = pd.read_csv('perceptron_toydata-truncated.txt', sep='\t')

In [118]:
X_train = df.drop(['label'], axis = 1).values
y_train = df[['label']].values

X_train_tensor = torch.from_numpy(X_train)
y_train_tensor = torch.from_numpy(y_train)

In [119]:
X_train = X_train_tensor.to(torch.float32)
y_train = y_train_tensor.to(torch.float32)

In [151]:
class Preceptron:
    def __init__(self, num_features, alpha = 1.):
        self.num_features = num_features
        self.weights= torch.rand(num_features) - 0.5
        self.bias = torch.tensor(0.)
        self.alpha = torch.tensor(alpha)
        
    def forward(self, x):
        
        weighted_sum_z = torch.dot(x, self.weights) + self.bias

        return torch.where(weighted_sum_z > torch.tensor(0.), torch.tensor(1.), torch.tensor(0.))

    def update(self, x, true_y: torch.tensor):
        prediction = self.forward(x)
        true_y = true_y[0]
        error = true_y - prediction
        error = torch.tensor(error)
        
        # update
        self.bias += error * self.alpha
        self.weights += error * x * self.alpha

        return error

In [152]:
def train(model, all_x, all_y, epochs):
    for epoch in range(epochs):
        error_count = 0

        for x, y in zip(all_x, all_y):
            error = model.update(x, y)
            error_count += abs(error)
    
        print(f"Ephocs {epoch + 1} errors = {error_count}")
        if error_count == 0:
            return

In [153]:
ppn = Preceptron(2, alpha = 0.1)

train(ppn, X_train, y_train, epochs=10)

Ephocs 1 errors = 3.0
Ephocs 2 errors = 1.0
Ephocs 3 errors = 0.0


/tmp/ipykernel_40303/3298152214.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  error = torch.tensor(error)


In [154]:
def compute_accuracy(model, all_x, all_y):
    correct = 0.
    for x, y in zip(all_x, all_y):
        predict = model.forward(x)
        correct += int(predict == y)

    return correct / len(all_x)

In [155]:
compute_accuracy(ppn, X_train, y_train)

1.0